<div style="font-size: 200%; font-weight: bold; color: maroon;">Procesado de la red de vuelos con GraphFrames (III)</div>
<!-- Paulo Villegas, 2021 - v. 2.0 -->

In [ ]:
spark.conf.get('spark.master')

# 1. Leer datos

Regeneramos el grafo desde los DataFrames grabados a disco

In [ ]:
import graphframes as gf

vertices = spark.read.parquet('OUT/flights-vertices')
edges = spark.read.parquet('OUT/flights-edges')

g = gf.GraphFrame(vertices, edges)

In [ ]:
g.vertices.count(), g.edges.count()

In [ ]:
g.edges.printSchema()

In [ ]:
g.vertices.printSchema()

# 2. Aggregate messages

## 2.1 Ejemplo de agregación de la guía de usuario
Repetimos aquí el ejemplo de algoritmo de agregación de mensajes de la [guía de GraphFrames](https://graphframes.github.io/user-guide.html#message-passing-via-aggregatemessages)

In [ ]:
from graphframes.lib import AggregateMessages as AM

In [ ]:
import pyspark.sql.functions as F

from graphframes.examples import Graphs
friends = Graphs(sqlContext).friends()  # Get example graph

In [ ]:
friends.vertices.toPandas()

In [ ]:
# For each user, sum the ages of the adjacent users.

# Send the age to the opposite node
msgToSrc = AM.dst["age"]
msgToDst = AM.src["age"]

# Sum all ages arriving to the node
agg = friends.aggregateMessages(
                F.sum(AM.msg).alias("summedAges"),
                sendToSrc=msgToSrc,
                sendToDst=msgToDst)
agg.toPandas()

## 2.2 Agregación en grafo de rutas
Volvemos a los aviones: vamos a calcular, para un aeropuerto dado, cuántas rutas aéreas distintas tengo desde allí haciendo una conexión (es decir, rutas áereas con dos saltos).

Para ello usamos el algoritmo de propagación de mensajes para sumar a cada nodo el grado de todos sus nodos adyacentes

In [ ]:
# Añadimos el grado de salida a cada uno de los vértices
vertices_deg = g.vertices.join(g.outDegrees, 'id')

In [ ]:
vertices_deg.printSchema()

In [ ]:
# Y creamos un grafo con el nuevo DataFrame
g2 = gf.GraphFrame(vertices_deg, edges)

In [ ]:
# Ahora pasamos a cada nodo el grado de todos sus vecinos, y los sumamos
import pyspark.sql.functions as F

hop2 = g2.aggregateMessages( F.sum(AM.msg).alias('hop2'), sendToSrc=AM.dst['outDegree'] )

lo que hemos hecho es:
* hace que cada nodo destino mande al nodo fuente su grado: `sendToSrc=AM.dst['degree']`
* sumar en el nodo todos los mensajes ( _grados_ ) que recibe: `F.sum(AM.msg)` (cambiamos el nombre de la columna a `hop2` para que quede más claro)

In [ ]:
hop2.limit(10).toPandas()

In [ ]:
vertices_hop2 = g2.vertices.select('id', 'Name', 'City', 'Country', 'outDegree').join(hop2, 'id')

In [ ]:
vertices_hop2.limit(10).toPandas()

### 2.2.1 Ejemplo

In [ ]:
EJEMPLO = 2142

# Veamos un nodo
vertices_hop2.filter(vertices_hop2.id == EJEMPLO).toPandas()

In [ ]:
# Y sus conexiones de salida
g.edges.filter(g.edges.src == EJEMPLO).toPandas()